<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Вывод</a></span></ul></div>

# Защита персональных данных клиентов

**Цель проекта**: защитить данные клиентов страховой компании - разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию, и обосновать корректность его работы.

Корректность выбранного алгоритма будет проверяться на моделях линейной регрессии с помощью коэффициента детерминации.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

data = pd.read_csv('/insurance.csv')
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


### Вывод
Таблица содержит следующие данные о застрахованных:
- пол
- возраст
- заработная плата
- количество членов в семье застрахованного
- факт осуществления страховой выплаты

В таблице 5000 строк, пропусков нет.

## Умножение матриц

Выделим из выборки обучающие и целевой признаки:

In [3]:
features = data.drop('Страховые выплаты', axis=1)
target = data['Страховые выплаты']
print('Размер матрицы с обучающими признаками:', features.shape)
print('Размер матрицы с целевым признаком:', target.shape)

Размер матрицы с обучающими признаками: (5000, 4)
Размер матрицы с целевым признаком: (5000,)


**Вопрос**: признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?

**Ответ**: качество линейной регрессии не изменится.

**Обоснование**: 

Обозначения:

- $X$ — матрица признаков

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $E$ - единичная матрица

- $w$ — вектор весов линейной регрессии (я сознательно опускаю здесь независимый член уравнения, поскольку даём общее обоснование, размерность матрицы признаков не имеет значения)

оптимальные коэффициенты регрессии рассчитваются по формуле $ w = (X^T X)^{-1} X^T y $

Покажем, что после умножения признаков на квадратную обратимую матрицу P размером n*n, где n - количество столбцов в матрице с признаками, коэффициенты $w_1 = P^{-1} w $: 
$$w_1 = ((XP)^T (XP))^{-1} (XP)^T y = (P^T X^T X P)^{-1} P^T X^T y = P^{-1} (X^T X)^{-1} P^T (P^T)^{-1} X^T y = P^{-1} (X^T X)^{-1} E X^T y = P^{-1} (X^T X)^{-1} X^T y = P^{-1} w$$

При этом предсказания новой модели будут равны предсказаниям исходной, т. к.:
$$a_1 = X P w_1 = X P P^{-1} w = X E w = X w$$

In [4]:
X = features
y = target
P = np.random.normal(size = (4, 4))

# коэффициенты исходной модели:
w = np.linalg.inv(X.T @ X) @ X.T @ y
print('Коэффициенты исходной модели:', np.array(w))

Коэффициенты исходной модели: [-4.43854686e-02  2.33356224e-02 -1.17739038e-05 -4.55168125e-02]


In [5]:
# умножаем матрицу признаков на обратимую матрицу:
X1 = X @ P

# коэффициенты модели после преобразования:
w1 = np.linalg.inv(X1.T @ X1) @ X1.T @ y
print('Коэффициенты преобразованной модели:', np.array(w1))

Коэффициенты преобразованной модели: [-0.1000635   0.01508085 -0.07919564  0.04526505]


Убедимся, что $w_1 = P^{-1} w $

In [6]:
np.linalg.inv(P) @ w

array([-0.1000637 ,  0.01508099, -0.07919594,  0.04526537])

In [7]:
a = X.values @ w
print('Предсказания исходной модели: \n \n', pd.Series(a), sep = '')

Предсказания исходной модели: 
 
0       0.282873
1       0.580513
2       0.429481
3      -0.091957
4       0.301713
          ...   
4995    0.142035
4996    0.130942
4997   -0.023457
4998   -0.052559
4999    0.085475
Length: 5000, dtype: float64


In [8]:
a1 = X1.values @ w1
print('Предсказания преобразованной модели: \n \n', pd.Series(a1), sep = '')

Предсказания преобразованной модели: 
 
0       0.282873
1       0.580514
2       0.429481
3      -0.091957
4       0.301713
          ...   
4995    0.142036
4996    0.130942
4997   -0.023456
4998   -0.052559
4999    0.085475
Length: 5000, dtype: float64


## Алгоритм преобразования

**Предлагаемый алгоритм защиты данных**: 
- сгенерируем случайную обратимую матрицу $p$ размера 4*4 
- проверим матрицу на обратимость
- умножим матрицу с признаками на матрицу $p$
- получим преобразованный массив признаков той же размерности, что и исходный
- построим модель линейной регрессии на исходных данных, получим предсказания, вычислим $R^2$
- построим модель линейной регрессии на преобразованных данных, получим предсказания, вычислим $R^2$
- сравним метрики и предсказания двух моделей

## Проверка алгоритма

Сгенерируем обратимую матрицу 4*4 и умножим признаки на неё:

In [9]:
# генерация обратимой матрицы 4*4
p = np.random.RandomState(1).normal(size = (4, 4))
p

array([[ 1.62434536, -0.61175641, -0.52817175, -1.07296862],
       [ 0.86540763, -2.3015387 ,  1.74481176, -0.7612069 ],
       [ 0.3190391 , -0.24937038,  1.46210794, -2.06014071],
       [-0.3224172 , -0.38405435,  1.13376944, -1.09989127]])

Проверим марицу на обратимость:

In [10]:
try: 
    np.linalg.inv(p)
except:
    print('Сгенерирована необратимая матрица')

In [11]:
features_chahged = features @ p
features_chahged.columns = features.columns
display(features_chahged.head())

,Пол,Возраст,Зарплата,Члены семьи
0,15861.122805,-12464.129521,72592.696557,-102216.361534
1,12162.971984,-9582.329103,55641.496718,-78321.462370
2,6724.917838,-5303.522507,30754.866219,-43285.029900
3,13321.459031,-10447.845079,61008.809561,-85926.052714
4,8352.776166,-6573.621640,38209.343715,-53792.059280


Теперь признаки в таблице исходных данных зашифрованы. Проверим, изменится ли качество линейной регрессии. 

Построим модели для исходных и преобразованных данных:

In [12]:
# функция для обучения модели и получения результатов
def LR (features, target):
    linear = LinearRegression ()
    linear.fit(features, target)
    predictions = linear.predict(features)
    print('R2 модели:', round(r2_score(target, predictions), 3), '\n')
    print('Предсказания модели: \n', pd.Series(predictions), '\n')
    print('Коэффициенты модели:', linear.intercept_, linear.coef_)

# модель на исходных данных
LR(features, target)

R2 модели: 0.425 

Предсказания модели: 
 0       0.511727
1       0.684316
2       0.093734
3      -0.222589
4       0.065084
          ...   
4995    0.028390
4996    0.253367
4997   -0.256970
4998   -0.190992
4999    0.049050
Length: 5000, dtype: float64 

Коэффициенты модели: -0.9382355041527256 [ 7.92580543e-03  3.57083050e-02 -1.70080492e-07 -1.35676623e-02]


In [13]:
# модель на зашифрованных данных
LR(features_chahged, target)

R2 модели: 0.425 

Предсказания модели: 
 0       0.511727
1       0.684316
2       0.093734
3      -0.222589
4       0.065084
          ...   
4995    0.028390
4996    0.253367
4997   -0.256970
4998   -0.190992
4999    0.049050
Length: 5000, dtype: float64 

Коэффициенты модели: -0.9382355041358567 [0.04432657 0.01894927 0.03689241 0.03075391]


Мы убедились, что качество модели после преобразования не изменилось: идентичны предсказания и коэффициенты $R^2$ двух моделей.
Можно также убедиться, что коэффициенты модели на зашифрованных данных можно получить с помощью умножения коэффицентов исходной модели на $ (P)^{-1} $

In [14]:
coef = np.array([7.92580543e-03,  3.57083050e-02, -1.70080492e-07, -1.35676623e-02])
np.linalg.inv(p) @ coef

array([0.04432657, 0.01894927, 0.03689241, 0.03075391])

## Вывод
Перед нами была поставлена задача разработки алгоритма защиты данных клиента, который не ухудшит качество модели линейной регрессии. 

В качестве алгоритма рекомендуется использовать умножение признаков модели на обратимую матрицу размера n*n, где n - количество предикторов в модели. Алгоритм обоснован математически и проверен на практике.
Предсказания модели, а соответственно, и метрики качества (на примере коэффициента детерминации $R^2$) при этом не меняются, а данные клиентов будут защищены. 